### Μεταφόρτωση αρχείων από το τοπικό σύστημα.

Με την βιβλιοθήκη <code>files</code> από το <code>google.colab</code> ανεβάζω τα απαραίτητα αρχεία για την υλοποίηση της άσκησης.

In [63]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving dataset-HAR-PUC-Rio.csv to dataset-HAR-PUC-Rio.csv
User uploaded file "dataset-HAR-PUC-Rio.csv" with length 14283211 bytes


# Data Preprocessing
> **Γιατί κάνουμε data preprocessing**: Το data set μας πολύ πιθανών να αποτελείται απο features διαφορετικών range.Για παράδειγμα, το ένα feature μπορεί να είναι η ηλικία ενός ατόμου που κυμαίνεται στο διάστημα $[0,120]$, ενώ ένα άλλο feature μπορεί να αναφέρεται σε κάποιο χρηματικό ποσό που κυμαίνεται στο διάστημα $[0,100000]$.Αυτό έχει ως αποτέλεσμα το δεύτερο feature να επηρεάζει περισσότερο την έξοδο από το πρώτο.Επομένως προσθέτουμε ένα bias ως προς το δεύτερο χαρακτηριστικό.<br><br>
>**Normalization**: Με αυτή την μέθοδο μεταφέρουμε το εύρος κάθε feature στο διάστημα $[0,1]$.Έτσι επιλύουμε το παραπάνω πρόβλημα.Αυτή η μέθοδος χρησιμοποιείται σε αλγόριθμους που δεν κάνουν υποθέσεις για την κατανομή των δεδομένων , όπως τα **neural networks**.Για αυτό τον λόγο θα χρησιμοποιηθεί αυτή η τεχνική για το πρόβλημά μας.<br><br>
>**Standarization**: Αυτή η μέθοδος υποθέτει ότι τα δεδομένα ακολουθούν gaussian distribution μετασχηματίζοντας τα έτσι ώστε να έχουν μηδενική μέση τιμή και διασπορά ίση με 1.Χρησιμοποιείται λοιπόν σε αλγόριθμους που υποθέτουν ότι τα δεδομένα ακολουθούν gaussian distribution όπως το **linear regresion**.<br><br>
>**Centralization**: Αυτή η μέθοδος μετασχηματίζει τα δεδομένα έτσι ώστε να έχουν μηδενική μέση τιμή. Παρόλα αυτά δεν λύνει το πρόβλημα που αναφέρθηκε παραπάνω.

In [69]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# read dataset
dataset = pd.read_csv("dataset-HAR-PUC-Rio.csv",delimiter=";", low_memory=False)

# create dictionary that maps strings to numbers
string_to_num = {'debora': 1, 'katia': 2, 'wallace': 3, 'jose_carlos': 4, 'Woman': 1, 'Man': 2, 'sitting': 1, 'sittingdown': 2, 'standing': 3, 'standingup': 4, 'walking':5 }

# lambda function
def map_strings_to_nums(col):
    return col.apply(lambda x: string_to_num[x] if x in string_to_num else x)

# apply lambda function to string columns
dataset[['user', 'gender', 'class']] = dataset[['user', 'gender', 'class']].apply(map_strings_to_nums)

# replace ',' with '.' so astype can work
dataset['how_tall_in_meters'] = dataset['how_tall_in_meters'].str.replace(',','.')
dataset['body_mass_index'] = dataset['body_mass_index'].str.replace(',','.')

# split features and Output
X = dataset.drop('class', axis=1)
Y = dataset['class']

# Normalize features
scaler = MinMaxScaler()
dataset_normalized = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Cross Validation

> **Γιατί;**: Αυτή η μέθοδος χρησιμοποιείται για να εξασφαλίσουμε ότι το score του αλγορίθμου μας δεν εξαρτάται από τον τρόπο που επιλέγουμε τα <code>train_set</code> και <code>test_set</code>.<br><br>
> **Πως**;: Χωρίζουμε το dataset σε Κ folds και χρησιμοποιούμε τα Κ-1 για <code>train_set</code> και το υπολοιπώμενο για <code>test_set</code> μέχρι και τα K folds να έχουν γίνει <code>test_set</code>.Στο παράδειγμά μας θα χρησιμοποιήσουμε 5-fold cross validation.